# Machine Learning now

In [1]:
import pandas as pd
import os
import numpy as np
import sklearn 
import scipy.linalg as scplinag
from sklearn.neighbors import KDTree
from scipy.spatial import distance

# Whole dataset and 50 NN 

Attributes as in Weinmann, 2013, plus relative height (Weinmann, 2014) 

In [13]:
# Define a data frame with all my data# Define  
FILE_PATH_ScenarioB = r"../DATA/ML_datasets/ScenarioB"
FILE_PATH = r"../DATA"
FILE_NAME = r"/5_Data_ML_attributes_10NN.txt"
IMAGE_FILE_PATH = r"images"
df = pd.read_csv(FILE_PATH+FILE_NAME, delimiter=',')
df.head()

,X,Y,Z,class,lambda1,lambda2,lambda3,lambda4,lambda5,lambda6,lambda7,lambda8,local_density,radius_neighbourhood,relative_height,verticality
0,1905.494751,21122.037109,38.884586,2,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,25877.928393,0.046644,-2.497189,0.000847
1,1905.502930,21122.035156,38.883110,2,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,46635.546755,0.038329,-2.498982,0.000847
2,1905.512939,21122.033203,38.883949,2,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,75836.463465,0.032594,-2.498448,0.000847
3,1905.520874,21122.031250,38.882137,2,0.350668,0.646708,0.002624,0.000033,0.997376,0.003784,0.000451,0.001588,82825.790636,0.031650,-2.500576,0.000847
4,1905.531128,21122.029297,38.882839,2,0.391125,0.603946,0.004929,0.000041,0.995071,0.003847,0.000460,0.003054,82835.520399,0.031649,-2.500191,0.000902


In [14]:
columns_to_drop_10 = ["X", "Y", "Z", "radius_neighbourhood", "relative_height"]

for elem in columns_to_drop_10:
    df.drop([elem], axis=1, inplace=True)

df.head()

,class,lambda1,lambda2,lambda3,lambda4,lambda5,lambda6,lambda7,lambda8,local_density,verticality
0,2,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,25877.928393,0.000847
1,2,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,46635.546755,0.000847
2,2,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,75836.463465,0.000847
3,2,0.350668,0.646708,0.002624,0.000033,0.997376,0.003784,0.000451,0.001588,82825.790636,0.000847
4,2,0.391125,0.603946,0.004929,0.000041,0.995071,0.003847,0.000460,0.003054,82835.520399,0.000902


### I have made a mistake with the class names, therefore I need to re-name them now

In the file 3_CreateFinalClasses, I did not change the names of the classes properly. Therefore, I will have to re-name them now. 

Manually, not to confuse it.

In [15]:
# Road (second most points)
df["class"].where(df["class"] != 2, 1, inplace=True)
# Sidewalk
df["class"].where(df["class"] != 3, 2, inplace=True)
# Curb
df["class"].where(df["class"] != 4, 3, inplace=True)
# Building (most points)
df["class"].where(df["class"] != 5, 4, inplace=True)
# Other pole like objects
df["class"].where(df["class"] != 6, 5, inplace=True)
# Small poles
df["class"].where(df["class"] != 7, 6, inplace=True)
# Pedestrians
df["class"].where(df["class"] != 11, 7, inplace=True)
# 2 wheelers
df["class"].where(df["class"] != 12, 8, inplace=True)
# 4 wheelers
df["class"].where(df["class"] != 13, 9, inplace=True)
# Trees
df["class"].where(df["class"] != 14, 10, inplace=True)
# Potted plants
df["class"].where(df["class"] != 15, 11, inplace=True)

In [16]:
df['class'].value_counts()

4     6359799
1     3230632
2      884203
9      318893
10     105205
3       59854
8       39803
7       23070
6       18024
11       8969
5        3842
Name: class, dtype: int64

In [17]:
df.drop(["class"], axis=1, inplace=True)
df.head()

,lambda1,lambda2,lambda3,lambda4,lambda5,lambda6,lambda7,lambda8,local_density,verticality
0,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,25877.928393,0.000847
1,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,46635.546755,0.000847
2,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,75836.463465,0.000847
3,0.350668,0.646708,0.002624,0.000033,0.997376,0.003784,0.000451,0.001588,82825.790636,0.000847
4,0.391125,0.603946,0.004929,0.000041,0.995071,0.003847,0.000460,0.003054,82835.520399,0.000902


# Rename the columns to make it easier

In [18]:
df = df.rename(index=str, columns={"lambda1": "lambda1_10NN"})
df = df.rename(index=str, columns={"lambda2": "lambda2_10NN"})
df = df.rename(index=str, columns={"lambda3": "lambda3_10NN"})
df = df.rename(index=str, columns={"lambda4": "lambda4_10NN"})
df = df.rename(index=str, columns={"lambda5": "lambda5_10NN"})
df = df.rename(index=str, columns={"lambda6": "lambda6_10NN"})
df = df.rename(index=str, columns={"lambda7": "lambda7_10NN"})
df = df.rename(index=str, columns={"lambda8": "lambda8_10NN"})
df = df.rename(index=str, columns={"local_density": "local_density_10NN"})
df = df.rename(index=str, columns={"verticality": "verticality_10NN"})

df.head()

,lambda1_10NN,lambda2_10NN,lambda3_10NN,lambda4_10NN,lambda5_10NN,lambda6_10NN,lambda7_10NN,lambda8_10NN,local_density_10NN,verticality_10NN
0,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,25877.928393,0.000847
1,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,46635.546755,0.000847
2,0.189010,0.808544,0.002447,0.000028,0.997553,0.003467,0.000408,0.001349,75836.463465,0.000847
3,0.350668,0.646708,0.002624,0.000033,0.997376,0.003784,0.000451,0.001588,82825.790636,0.000847
4,0.391125,0.603946,0.004929,0.000041,0.995071,0.003847,0.000460,0.003054,82835.520399,0.000902


## Now save that in scenario B folder

In [19]:
df.to_csv(FILE_PATH_ScenarioB+'/ScenarioB_10NNFeatures.txt', index= False)

In [20]:
# Clean variables I domnt need right now

df = None
corr_matrix = None
X = None
y = None
data = None 